In [1]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install statsmodels
!{sys.executable} -m pip install googlemaps

    100% |████████████████████████████████| 15.0MB 10kB/s eta 0:00:011
    100% |████████████████████████████████| 16.7MB 25kB/s eta 0:00:011
    100% |████████████████████████████████| 512kB 380kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 753kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 949kB/s eta 0:00:01
  Running setup.py bdist_wheel for subprocess32 ... done
  Stored in directory: /home/students/2018/jason.kao/.cache/pip/wheels/7d/4c/a4/ce9ceb463dae01f4b95e670abd9afc8d65a45f38012f8030cc
Successfully built subprocess32
    100% |████████████████████████████████| 24.3MB 12kB/s eta 0:00:011
    100% |████████████████████████████████| 6.2MB 61kB/s eta 0:00:011
    100% |████████████████████████████████| 46.7MB 6.3kB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 2.2MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 1.9MB/s eta 0:00:01
    100% 

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import statsmodels.api as sm

In [3]:
flights = pd.read_csv('./data/flights/all_flights.csv')
flights = flights.drop(columns=['FL_NUM', 'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID'])
flights = flights.loc[:, ~flights.columns.str.contains('^Unnamed')]

ufos = pd.read_csv("data/ufo-sightings/ufos-2013.csv", sep=',', low_memory=False)
ufos['lat'] = ufos['latitude']
ufos['lng'] = ufos['longitude']
ufos = ufos[[ 'datetime', 'duration (seconds)', 'lat', 'lng' ]]
ufos = ufos[ ufos['datetime'] > '2013-01-01 00:00:00' ]

In [ ]:
for _, u in ufos.iterrows():
    for _, f in flights.iterrows():
        d_to_origin = (( (f['o_lat'] - u['lat'])**2 + (f['o_lng'] - u['lng'])**2 ) ** 0.5) * 55.2428
        d_to_dest = (( (f['d_lat'] - u['lat'])**2 + (f['d_lng'] - u['lng'])**2 ) ** 0.5) * 55.2428
        if d_to_origin < 3000:
            print 'caught origin: {}{}'.format(u, f)
            break
        elif d_to_dest < 3000:
            print 'caught dest: {}{}'.format(u, f)
            break
    break

In [39]:
def flight_in_range(row):
    return 1
ufos['flight'] = ufos.apply(flight_in_range)
ufos.head()

,datetime,duration (seconds),lat,lng
5,2013-01-01 00:01:00,180.0,45.523611,-122.675000
6,2013-01-01 00:02:00,120.0,30.411111,-88.827778
7,2013-01-01 00:02:00,60.0,18.473833,-66.938512
8,2013-01-01 00:04:00,300.0,-27.522349,153.191323
9,2013-01-01 00:04:00,180.0,37.639167,-120.995833


In [40]:
flights.head()

,FL_DATE,DEP_TIME,ARR_TIME,o_lat,o_lng,d_lat,d_lng
0,2013-01-01,71820,82680,40.690292,-73.988544,26.640628,-81.872308
1,2013-01-01,43140,53940,40.690292,-73.988544,26.640628,-81.872308
2,2013-01-01,56640,67140,40.690292,-73.988544,26.640628,-81.872308
3,2013-01-01,59580,68820,26.640628,-81.872308,40.690292,-73.988544
4,2013-01-01,70140,79560,26.640628,-81.872308,40.690292,-73.988544
